<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MMLU_GEMINI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Apr 21 15:10:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   65C    P8              16W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q transformers
!pip install -q tqdm
!pip install -q pandas
!pip install -q tensor_parallel
!pip install -q argparse
!pip install -q einops
!pip install -q accelerate
#!pip install -q torch==2.0.0+cu118
!pip install -q torch

!pip install colab-env --upgrade -q
!pip install openai -q

!pip install datasets -q
!pip install utils -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
!git clone https://github.com/FranxYao/chain-of-thought-hub.git

Cloning into 'chain-of-thought-hub'...
remote: Enumerating objects: 3043, done.
remote: Counting objects: 100% (326/326), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 3043 (delta 178), reused 236 (delta 168), pack-reused 2717
Receiving objects: 100% (3043/3043), 8.30 MiB | 15.43 MiB/s, done.
Resolving deltas: 100% (964/964), done.


In [4]:
import colab_env
import os
import openai
import IPython
import pytz
from datetime import datetime
import json
from pathlib import Path

Mounted at /content/gdrive


In [5]:
def test_answer_mmlu_(pred_str, ans):
    pattern = 'the answer is ('
    pred = pred_str.lower().split(pattern)

    if(len(pred) > 1):
        # print(pred)
        pred = pred[1][0]
        gold = ans.lower()
        # print('debug 1, pred %s, gold %s' % (pred, gold))
        return pred == gold
    else:
        pred = 'C'
        # print(ans_str)
        gold = ans.lower()
        # print('debug 2, pred %s, gold %s' % (pred, gold))
        return pred == gold

# extract answer in pred_str and compare with ans_str
def test_answer_mmlu_claude_instant(pred_str, ans_str):
    pattern = 'the answer is '
    pred = pred_str.lower().split(pattern)
    if len(pred) == 1:
        return False
    else:
        return pred[1][0] == ans_str.lower()

def test_answer_mmlu_claude(pred_str, ans_str):
    pattern = 'the answer is '
    pred = pred_str.lower().split(pattern)

    if(len(pred) > 1):
        # print(pred)
        pred = pred[1]
        for p in pred:
            if(p.isalpha()): break
        pred = p
        print(ans_str)
        gold = ans_str.lower()
        print('debug 1, pred %s, gold %s' % (pred, gold))
        return pred == gold
    else:
        pred = 'c'
        # print(ans_str)
        gold = ans_str.lower()
        # print('debug 2, pred %s, gold %s' % (pred, gold))
        return pred == gold

def test_answer_mmlu(pred_str, ans_str):
    pattern = 'the answer is ('
    pred = pred_str.lower().split(pattern)

    if(len(pred) > 1):
        # print(pred)
        pred = pred[1][0]
        gold = ans_str.split('A:\n')[1][0].lower()
        # print('debug 1, pred %s, gold %s' % (pred, gold))
        return pred == gold
    else:
        pred = 'C'
        # print(ans_str)
        gold = ans_str.split('A:\n')[1][0].lower()
        # print('debug 2, pred %s, gold %s' % (pred, gold))
        return pred == gold

def parse_pred_ans(filename):
    with open(filename) as fd: lines = fd.readlines()
    am, a = None, None
    num_q, acc = 0, 0
    current_mode = 'none'
    questions = []
    ans_pred = []
    ans_gold = []
    for l in lines:
        if(l.startswith('Q: ')):
            if(am is not None and a is not None):
                questions.append(q)
                ans_pred.append(am)
                ans_gold.append(a)
                # print(am)
                # print(a)
                if(test_answer_mmlu(am, a)):
                    acc += 1
            current_mode = 'q'
            q = l
            num_q += 1
        elif(l.startswith('A_model:')):
            current_mode = 'am'
            am = l
        elif(l.startswith('A:') and not l.startswith("A: Let's think step by step")):
            current_mode = 'a'
            a = l
        else:
            if(current_mode == 'q'): q += l
            elif(current_mode == 'am'): am += l
            elif(current_mode == 'a'): a += l
            else:
                raise ValueError(current_mode)

    questions.append(q)
    ans_pred.append(am)
    ans_gold.append(a)
    # print(am)
    # print(a)
    if(test_answer_mmlu(am, a)):
        acc += 1
    print('num_q %d correct %d ratio %.4f' % (num_q, acc, float(acc / num_q)))
    return questions, ans_pred, ans_gold

def test_finished(ans_model):
    if('answer is' in ans_model): return True
    else: return False

def extract_ans(ans_model):
    ans_model = ans_model.split('\n')
    ans = []
    residual = []
    for li, al in enumerate(ans_model):
        ans.append(al)
        if('answer is' in al):
            break
    residual = list(ans_model[li + 1:])
    ans = '\n'.join(ans)
    residual = '\n'.join(residual)
    return ans, residual

In [6]:
TASKSTEST0old = [
        'anatomy',
        'college_biology',
        'college_chemistry',
        'college_computer_science',
        'college_mathematics',
        'college_medicine',
        'college_physics',
        'computer_security',
        'electrical_engineering',
        'machine_learning',
]


TASKSTEST = [
        'college_computer_science',
        'electrical_engineering',
        'machine_learning',
]

TASKS911 = [
        'anatomy',
        'college_biology',
        'college_chemistry',
        'college_computer_science',
        'college_mathematics',
        'college_medicine',
        'college_physics',
        'computer_security',
        'conceptual_physics',
        'econometrics',
        'electrical_engineering',
        'elementary_mathematics',
        'formal_logic',
        'global_facts',
        'high_school_biology',
        'high_school_chemistry',
        'high_school_computer_science',
        'high_school_european_history',
        'high_school_geography',
        'high_school_government_and_politics',
        'high_school_macroeconomics',
        'high_school_mathematics',
        'high_school_microeconomics',
        'high_school_physics',
        'high_school_psychology',
        'high_school_statistics',
        'high_school_us_history',
        'high_school_world_history',
        'public_relations',
        'security_studies',
        'sociology',
        'us_foreign_policy',
        'virology',
        'machine_learning',
        'world_religions']

GEMINI

In [7]:
!pip install google-generativeai -q

In [8]:
# Used to securely store your API key
from google.colab import userdata

In [9]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [10]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GEMINI')
genai.configure(api_key=GOOGLE_API_KEY)

In [11]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [12]:
#model = genai.GenerativeModel('gemini-1.0-pro-latest')
model = genai.GenerativeModel('gemini-1.5-pro-latest')
#query = "I bought a computer for 1200, repurchased it for 1600. how much did I earn? Take in consideration the money for the repurchased too."

#del query
query = "I bought a computer for $900, sold it for $1200, repurchased it for $1300, and sold it again for $1600. how much did I earn? Take in consideration the money for the repurchased too."
chat_response=model.generate_content(query)


print()
print("-" * 80)
print('Question: %s'%query)
print("-" * 80)

print()
print('Answer: ')
print(chat_response.text)
print()


--------------------------------------------------------------------------------
Question: I bought a computer for $900, sold it for $1200, repurchased it for $1300, and sold it again for $1600. how much did I earn? Take in consideration the money for the repurchased too.
--------------------------------------------------------------------------------

Answer: 
## Calculating Your Earnings

Here's how to break down your earnings:

**First Sale:**

* Sold for $1200
* Bought for $900
* Profit: $1200 - $900 = $300

**Second Sale:**

* Sold for $1600
* Bought for $1300 
* Profit: $1600 - $1300 = $300

**Total Earnings:**

* Combining profits from both sales: $300 + $300 = $600

Therefore, you earned **$600** from the two sales. 




In [13]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')
query = "I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering."

#del query
#query = "I bought a computer for $900, sold it for $1200, repurchased it for $1300, and sold it again for $1600. how much did I earn? Take in consideration the money for the repurchased too."
chat_response=model.generate_content(query)


print()
print("-" * 80)
print('Question: %s'%query)
print("-" * 80)

print()
print('Answer: ')
print(chat_response.text)
print()


--------------------------------------------------------------------------------
Question: I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.
--------------------------------------------------------------------------------

Answer: 
## Figuring out your change:

**1. Find the total cost of the ice cream cones:**  Since each cone costs $1.25 and you bought 6, we need to multiply the cost per cone by the number of cones:  $1.25 x 6 = $7.50

**2. Subtract the total cost from the amount you paid:** You gave the cashier a $10 bill, so we need to subtract the cost of the cones from that amount to find your change: $10.00 - $7.50 = $2.50

Therefore, you got **$2.50** back in change. 




In [14]:
#models/gemini-1.0-pro
#models/gemini-1.0-pro-001
#models/gemini-1.0-pro-latest
#models/gemini-1.0-pro-vision-latest
#models/gemini-1.5-pro-latest
#models/gemini-pro
#models/gemini-pro-vision

#model = genai.GenerativeModel('gemini-pro')
model_name = 'gemini-1.0-pro-latest'
model = genai.GenerativeModel(model_name)

#model = genai.GenerativeModel('gemini-1.5-pro-latest')

In [15]:
print(model)

 genai.GenerativeModel(
   model_name='models/gemini-1.0-pro-latest',
   generation_config={}.
   safety_settings={}
)


In [16]:
%mkdir /content/outputs/
import re
import time
import json

import numpy as np

from tqdm import tqdm
from datasets import load_dataset
from tenacity import retry, stop_after_attempt, wait_chain, wait_fixed

def main(tasks=TASKSTEST):
    #openai.api_key = openai.api_key
    mmlu_prompt = json.load(open('/content/chain-of-thought-hub/MMLU/lib_prompt/mmlu-cot.json'))
    for task in tasks:

        print()
        print('Testing %s ...' % task)
        print()

        i = 0
        acc = 0
        task_data = load_dataset("lukaemon/mmlu", task, trust_remote_code=True)
        #model="mistral-large-latest"

        #model = "open-mixtral-8x22b"

        with open('/content/outputs/test_%s_%s.txt' % (model_name, task), 'w') as fd:
        #with open('/content/outputs/test_gpt_3.5_turbo_%s.txt' % task, 'w') as fd:
            for q_ in tqdm(task_data['test'], total=len(task_data['test'])):
                q = q_['input'] + '\n'
                for letter in ['A', 'B', 'C', 'D']:
                    q += '(' + letter + ') ' + q_[letter] + ' '
                q += "\nA: Let's think step by step."

                prompt_q = mmlu_prompt[task] + "\n\n" + q
                #print(prompt_q)

                ### ADDED by Frank Morales 19/04/2024
                response=model.generate_content(prompt_q)

                #print(response.text)


                ### ADDED by Frank Morales 19/04/2023
                #ans_model = response.text

                try:
                     #x = int(input("Please enter a number: "))
                     #break
                     ans_model = response.text
                     ans_, residual = extract_ans(ans_model)
                except ValueError:
                     print("Oops!  That was no valid response.  Try again...")
                     print(prompt_q)
                     #print(response)


                del prompt_q
                del response
                ans_, residual = extract_ans(ans_model)

                a = q_['target']
                #print(a)
                fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
                i += 1

                if(test_answer_mmlu_(ans_, a)): acc += 1
            print('%s acc %.4f' % (task, acc / len(task_data['test'])))
        # write accuracy to file
        with open('/content/outputs/test_%s_%s_acc.txt' % (model_name, 'mmlu'), 'a') as fd:
          fd.write('%s acc %.4f\n' % (task, acc / len(task_data['test'])))

    # write average accuracy to file
    acc_list = []
    #with open('/content/outputs/test_%s_%s_acc.txt' % (model, args.prompt_type), 'r') as fd2:

    # with open('/content/outputs/test_%s_%s_acc.txt' % (model, 'multiple'), 'r') as fd2:
    with open('/content/outputs/test_%s_%s_acc.txt' % (model_name, 'mmlu'), 'r') as fd2:
        for line in fd2:
            acc_list.append(float(line.split(' ')[2]))
    with open('/content/outputs/test_%s_%s_acc.txt' % (model_name, 'mmlu'), 'a') as fd:
        fd.write('Average acc %.4f\n' % (np.mean(acc_list)))
    return

if __name__ == '__main__':
    main()


Testing college_computer_science ...



Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/5 [00:00<?, ? examples/s]

100%|██████████| 100/100 [07:34<00:00,  4.54s/it]


...

Generating test split:   0%|          | 0/112 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/5 [00:00<?, ? examples/s]

100%|██████████| 112/112 [06:04<00:00,  3.26s/it]

machine_learning acc 0.5625


https://medium.com/@frankmorales_91352/gemini-llm-unveiling-the-future-of-language-models-2eea7aa2c550

MODEL: mistral-large-latest

college_computer_science acc 0.5200
electrical_engineering acc 0.6069
machine_learning acc 0.5982
Average acc 0.5750


MODEL: open-mixtral-8x22b

college_computer_science acc 0.2300
electrical_engineering acc 0.3310
machine_learning acc 0.2321
Average acc 0.2644


MODEL: open-mixtral-8x22b-2404

college_computer_science acc 0.1900
electrical_engineering acc 0.3310
machine_learning acc 0.2232
Average acc 0.2481


MODEL: meta-llama/Meta-Llama-3-8B-Instruct

college_computer_science acc 0.3300
electrical_engineering acc 0.2414
machine_learning acc 0.3125
Average acc 0.2946

MODEL: gemini-pro (trial-1)

college_computer_science acc 0.4600
electrical_engineering acc 0.7241
machine_learning acc 0.4911
Average acc 0.5584

MODEL: gemini-pro (trial-2)

college_computer_science acc 0.4600
electrical_engineering acc 0.6759
machine_learning acc 0.4821
Average acc 0.5393

MODEL: gemini-pro (trial-3)

college_computer_science acc 0.4900
electrical_engineering acc 0.7034
machine_learning acc 0.4911
Average acc 0.5615

MODEL: gemini-1.0-pro-latest (trial-1)

college_computer_science acc 0.4500
electrical_engineering acc 0.6759
machine_learning acc 0.5179
Average acc 0.5479


MODEL: gemini-1.0-pro-latest

college_computer_science acc 0.5300
electrical_engineering acc 0.7172
machine_learning acc 0.5625
Average acc 0.6032

In [17]:
#model="mistral-large-latest"
#model = "open-mixtral-8x22b"

print()
print('MODEL: %s'%model_name)
print()
acc_file='/content/outputs/test_%s_%s_acc.txt' % (model_name, 'mmlu')
#print(acc_file)

with open('/content/outputs/test_%s_%s_acc.txt' % (model_name, 'mmlu'), 'r') as fd:
     for line in fd:
            print(line)
            #acc_list.append(float(line.split(' ')[2]))


MODEL: gemini-1.0-pro-latest

college_computer_science acc 0.5300

electrical_engineering acc 0.7172

machine_learning acc 0.5625

Average acc 0.6032

